# Initialisation des librairies et du fichier

In [1]:
# FACULTATIF
# à utiliser dans google colab
# uploader un fichier local depuis l'ordinateur

from google.colab import files
uploaded = files.upload()


Saving eco2mix-regional-tr.csv to eco2mix-regional-tr.csv


In [2]:
# Import des bibliothèques nécessaires
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.core.dtypes.common import is_numeric_dtype

# Lecture du fichier CSV brut (période 2023-2025)
df = pd.read_csv("eco2mix-regional-tr.csv",
                 sep=';',                   # séparateur confirmé
                 na_values=['ND', '-'],     # gestion des valeurs manquantes
                 low_memory=False,          # éviter erreurs de type
                 encoding='utf-8')

# Options d’affichage dans le notebook
pd.set_option('display.float_format', lambda x: '%.4f' % x)  # afficher les floats avec 4 décimales
pd.set_option('display.max_columns', None)                   # afficher toutes les colonnes, même si nombreuses

# Vérification du résultat
df.head()


,Code INSEE région,Région,Nature,Date,Heure,Date - Heure,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Pompage (MW),Bioénergies (MW),Ech. physiques (MW),Stockage batterie,Déstockage batterie,TCO Thermique (%),TCH Thermique (%),TCO Nucléaire (%),TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%),Column 68
0,24,Centre-Val de Loire,Données temps réel,2023-05-16,04:15,2023-05-16T04:15:00+02:00,1361.0000,0.0000,7742.0000,504.0000,0.0000,18.0000,0.0000,52.0000,-6956.0000,NaN,NaN,0.0000,0.0000,568.8500,66.5700,37.0300,31.5400,0.0000,0.0000,1.3200,19.5700,3.8200,58.4300,NaN
1,24,Centre-Val de Loire,Données temps réel,2023-05-16,06:15,2023-05-16T06:15:00+02:00,1597.0000,0.0000,7803.0000,609.0000,19.0000,24.0000,0.0000,52.0000,-6910.0000,NaN,NaN,0.0000,0.0000,488.6000,67.0900,38.1300,38.1100,1.1900,2.4300,1.5000,26.0900,3.2600,58.4300,NaN
2,24,Centre-Val de Loire,Données temps réel,2023-05-16,06:30,2023-05-16T06:30:00+02:00,1623.0000,0.0000,7805.0000,631.0000,21.0000,25.0000,0.0000,52.0000,-6911.0000,NaN,NaN,0.0000,0.0000,480.9000,67.1100,38.8800,39.4900,1.2900,2.6900,1.5400,27.1700,3.2000,58.4300,NaN
3,24,Centre-Val de Loire,Données temps réel,2023-05-16,09:15,2023-05-16T09:15:00+02:00,1890.0000,0.0000,7799.0000,682.0000,193.0000,24.0000,0.0000,52.0000,-6860.0000,NaN,NaN,0.0000,0.0000,412.6500,67.0600,36.0800,42.6800,10.2100,24.6800,1.2700,26.0900,2.7500,58.4300,NaN
4,24,Centre-Val de Loire,Données temps réel,2023-05-16,10:00,2023-05-16T10:00:00+02:00,1801.0000,0.0000,7741.0000,844.0000,281.0000,22.0000,0.0000,52.0000,-7139.0000,NaN,NaN,0.0000,0.0000,429.8200,66.5600,46.8600,52.8200,15.6000,35.9300,1.2200,23.9100,2.8900,58.4300,NaN


# Gestion des types de données

In [3]:
# Conversion des colonnes temporelles au bon format

# On transforme la colonne 'Date' (texte comme "2013-01-01") en vrai objet date pour pouvoir trier, grouper, extraire l'année, etc.
df['Date'] = pd.to_datetime(df['Date'])

# La colonne 'Date - Heure' contient déjà une date et une heure combinées (ex : "2012-12-31T23:00:00+00:00"),
# donc on peut directement la convertir en format datetime
df['Date - Heure'] = pd.to_datetime(df['Date - Heure'])

# La colonne 'Heure' contient uniquement l’heure (ex : "00:00"), on la convertit en objet time
# On ajoute 'errors="coerce"' pour éviter les erreurs en cas de format inattendu
df['Heure'] = pd.to_datetime(df['Heure'], format='%H:%M', errors='coerce').dt.time

# On convertit le code INSEE en chaîne de caractères, car c’est un identifiant (et non un nombre à calculer)
df['Code INSEE région'] = df['Code INSEE région'].astype('str')

# On crée une nouvelle colonne indiquant le jour de la semaine (0 = lundi, 6 = dimanche), à partir de la colonne 'Date'
df['JourSemaine'] = df['Date'].dt.weekday

# Aperçu global du DataFrame : types de colonnes et valeurs manquantes
df.info()



/tmp/ipython-input-3-2482310693.py:8: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date - Heure'] = pd.to_datetime(df['Date - Heure'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002240 entries, 0 to 1002239
Data columns (total 31 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   Code INSEE région    1002240 non-null  object        
 1   Région               1002240 non-null  object        
 2   Nature               1002240 non-null  object        
 3   Date                 1002240 non-null  datetime64[ns]
 4   Heure                1002240 non-null  object        
 5   Date - Heure         1002240 non-null  object        
 6   Consommation (MW)    1001498 non-null  float64       
 7   Thermique (MW)       1001807 non-null  float64       
 8   Nucléaire (MW)       979733 non-null   float64       
 9   Eolien (MW)          1001827 non-null  float64       
 10  Solaire (MW)         1001826 non-null  float64       
 11  Hydraulique (MW)     1001811 non-null  float64       
 12  Pompage (MW)         975319 non-null   float64       
 1

# Nettoyage des lignes et colonnes vides

In [4]:
# Supprimer la colonne 'Column 68', qui est vide (0 valeurs non-nulles)
df = df.drop(columns=['Column 68'])

# Autre possibilité :
# Supprimer toutes les colonnes entièrement vides (ex : 'Column 68')
# df = df.dropna(axis=1, how='all')

# Gestion des doublons

In [5]:
# Vérifie le nombre de lignes en double dans le DataFrame
# Cela permet de savoir s’il y a des doublons exacts (mêmes valeurs sur toutes les colonnes)
nb_doublons = df.duplicated().sum()
print(str(nb_doublons) + " doublons détectés")

# Suppression des lignes dupliquées (si nb_doublons > 0)
# drop_duplicates() garde la première occurrence de chaque doublon
df_clean = df.drop_duplicates()

0 doublons détectés


# Gestion des valeurs manquantes

---



In [6]:
# Identification des valeurs manquantes
pd.DataFrame({
    "Nb valeurs manquantes" : df.isna().sum(),
    "% valeurs manquantes" : df.isna().mean() * 100})

,Nb valeurs manquantes,% valeurs manquantes
Code INSEE région,0,0.0000
Région,0,0.0000
Nature,0,0.0000
Date,0,0.0000
Heure,0,0.0000
Date - Heure,0,0.0000
Consommation (MW),742,0.0740
Thermique (MW),433,0.0432
Nucléaire (MW),22507,2.2457
Eolien (MW),413,0.0412


In [7]:
# Traitement des NaN par interpolation temporelle

# Trier par date avant interpolation
df_clean = df_clean.sort_values(by=['Date','Région'])

# Interpolation linéaire temporelle sur toutes les colonnes numériques
df_clean = df_clean.interpolate(method='linear')

# Vérifie s'il reste des NaN après interpolation
print(df_clean.isna().sum())


/tmp/ipython-input-7-4116043845.py:7: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_clean = df_clean.interpolate(method='linear')


Code INSEE région           0
Région                      0
Nature                      0
Date                        0
Heure                       0
Date - Heure                0
Consommation (MW)           0
Thermique (MW)              0
Nucléaire (MW)              0
Eolien (MW)                 0
Solaire (MW)                0
Hydraulique (MW)            0
Pompage (MW)                0
Bioénergies (MW)            0
Ech. physiques (MW)         0
Stockage batterie      194688
Déstockage batterie    194688
TCO Thermique (%)           0
TCH Thermique (%)           0
TCO Nucléaire (%)           0
TCH Nucléaire (%)           0
TCO Eolien (%)              0
TCH Eolien (%)              0
TCO Solaire (%)             0
TCH Solaire (%)             0
TCO Hydraulique (%)         0
TCH Hydraulique (%)         0
TCO Bioénergies (%)         0
TCH Bioénergies (%)         0
JourSemaine                 0
dtype: int64


In [8]:
# Les colonnes 'Stockage batterie' et 'Déstockage batterie' avaient été supprimées du premier fichier
# On choisit de les supprimer ici aussi afin de garder la cohérence entre les 2

df_clean = df_clean.drop(columns = ['Stockage batterie', 'Déstockage batterie'])

In [9]:
# Vérification des colonnes contenant encore des NaN

# On cherche les colonnes ayant encore des valeurs manquantes
missing = df_clean.isna().sum()
print("Colonnes avec NaN restants (non traités par interpolation) :")
print(missing[missing > 0].sort_values(ascending=False))

Colonnes avec NaN restants (non traités par interpolation) :
Series([], dtype: int64)


# Gestion des valeurs aberrantes

## Pourcentages

In [10]:
# Les colonnes TCO et TCH (%) représentent des %
# les colonnes TCO représentent la couverture des besoin d'une région par une filière.
# On considère qu'elles peuvent être > 100 % quand la filière produit plus que la consommation de la région

# les TCH en revanche doivent rester entre 0 et 100 % car elles représentent la production de la filière par rapport à sa capacité
col_TCH = ['TCH Nucléaire (%)','TCH Thermique (%)', 'TCH Eolien (%)', 'TCH Hydraulique (%)','TCH Solaire (%)', 'TCH Bioénergies (%)']

#Nombre de lignes ayant des valeurs > 100 par colonnes
display((df_clean[col_TCH] > 100).sum())

,0
TCH Nucléaire (%),2972
TCH Thermique (%),0
TCH Eolien (%),1167
TCH Hydraulique (%),158576
TCH Solaire (%),1646
TCH Bioénergies (%),92305


In [11]:
#remplacer les valeurs > 100 par 100
for col in col_TCH:
    df_clean[col] = df_clean[col].apply(lambda x: 100 if x > 100 else x)

# vérifier que le nombre de valeurs supérieures à 100 est maintenant à 0
display((df_clean[col_TCH] > 100).sum())

,0
TCH Nucléaire (%),0
TCH Thermique (%),0
TCH Eolien (%),0
TCH Hydraulique (%),0
TCH Solaire (%),0
TCH Bioénergies (%),0


In [12]:
# on vérifie que pour les colonnes TCO et TCH, il n'y a pas des valeurs < 0
col_TCH_TCO = ['TCH Nucléaire (%)','TCH Thermique (%)', 'TCH Eolien (%)', 'TCH Hydraulique (%)','TCH Solaire (%)', 'TCH Bioénergies (%)',
               'TCO Nucléaire (%)','TCO Thermique (%)', 'TCO Eolien (%)', 'TCO Hydraulique (%)','TCO Solaire (%)', 'TCO Bioénergies (%)']
display((df_clean[col_TCH_TCO] < 0).sum())

,0
TCH Nucléaire (%),0
TCH Thermique (%),0
TCH Eolien (%),0
TCH Hydraulique (%),0
TCH Solaire (%),0
TCH Bioénergies (%),0
TCO Nucléaire (%),0
TCO Thermique (%),0
TCO Eolien (%),0
TCO Hydraulique (%),0


In [13]:
# Interprétation :
# Les TCH au-delà de 100% ont été ramenés à 100%, car cela indique un taux supérieur à la capacité théorique maximale,
# ce qui n'est pas réaliste. Ces valeurs peuvent être dues à des erreurs de mesure.


## Valeurs Négatives

In [14]:
# On considère que les valeurs de production ne peuvent pas être négatives.
col_production = ['Thermique (MW)', 'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)', 'Hydraulique (MW)', 'Bioénergies (MW)']

# on vérifie la présence de valeurs négatives
display((df_clean[col_production] < 0).sum())

,0
Thermique (MW),0
Nucléaire (MW),0
Eolien (MW),0
Solaire (MW),0
Hydraulique (MW),0
Bioénergies (MW),0


# Statistiques descriptives

In [15]:
# Statistiques descriptives
df_clean.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,1002240,2024-04-10 12:00:00,2023-02-01 00:00:00,2023-09-06 00:00:00,2024-04-10 12:00:00,2024-11-14 00:00:00,2025-06-19 00:00:00,NaN
Consommation (MW),1002240.0000,4087.7023,0.0000,2478.0000,3792.0000,5229.0000,18657.0000,1992.2202
Thermique (MW),1002240.0000,224.6443,0.0000,17.0000,72.0000,259.0000,2642.0000,374.0531
Nucléaire (MW),1002240.0000,3454.0158,0.0000,0.0000,2622.0000,6375.2500,13326.0000,3631.4638
Eolien (MW),1002240.0000,442.4554,0.0000,66.0000,215.0000,549.0000,5444.0000,653.5104
Solaire (MW),1002240.0000,235.0622,0.0000,0.0000,17.0000,221.0000,4371.0000,479.6114
Hydraulique (MW),1002240.0000,605.5085,0.0000,13.0000,44.0000,896.0000,8039.0000,1024.8489
Pompage (MW),1002240.0000,-69.2669,-2358.0000,0.0000,0.0000,0.0000,0.0000,235.7144
Bioénergies (MW),1002240.0000,69.8041,0.0000,24.0000,58.0000,102.0000,298.0000,56.6076
Ech. physiques (MW),1002240.0000,-696.9941,-11945.0000,-4071.0000,360.0000,1868.0000,14579.0000,4102.6760


Le jeu de données est maintenant nettoyé (colonnes inutiles supprimées), corrigé (types datetime convertis), sans doublons, sans valeurs manquantes (par interpolation ou remplissage par 0).

## Ajout de colonnes pertinentes pour l'analyse

In [16]:
# Fonction permettant la création d'un dictionnaire pour réaliser l'aggrégation par jour sur les colonnes de type numérique

def getdico(df):
    dico={}
    for i in df.columns.values:
        if (is_numeric_dtype(df[f'{i}'])) and (i !='Code INSEE région') and (i != 'JourSemaine'):
            if 'TCO' in i or 'TCH' in i :
                dico[i]='mean'
            else:
               dico[i]='sum'
    return dico

In [21]:
# ajout de colonnes PROD et CONSO totale
df_clean['PROD']=(df_clean['Thermique (MW)']
                  + df_clean['Nucléaire (MW)']
                  + df_clean['Eolien (MW)']
                  + df_clean['Solaire (MW)']
                  + df_clean['Hydraulique (MW)']
                  + df_clean['Bioénergies (MW)'])

# le pompage est en valeurs négatives, on le remet en valeur absolue
df_clean['Pompage (MW)'] = abs(df_clean['Pompage (MW)'])
# Cette ligne doit être placée avant le calcul de CONSO. Sinon, la colonne CONSO pourrait contenir une valeur négative de pompage, ce qui biaiserait la consommation réelle.

# calculer la colonne CONSO
df_clean['CONSO']=df_clean['Consommation (MW)'] + df_clean['Pompage (MW)']

In [22]:
# Création de l'aggrégat par jour
df_clean=df_clean.groupby(['Code INSEE région','Région','Date', 'JourSemaine']).agg(getdico(df_clean))
df_clean=df_clean.reset_index()

In [23]:
# ajout de colonne Année-mois, mois et année
df_clean['Année'] = df_clean['Date'].dt.year
df_clean['Mois'] = df_clean['Date'].dt.month
df_clean['Année_mois'] = df_clean['Date'].dt.to_period('M').astype(str)

In [24]:
df_clean.head()

,Code INSEE région,Région,Date,JourSemaine,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Pompage (MW),Bioénergies (MW),Ech. physiques (MW),TCO Thermique (%),TCH Thermique (%),TCO Nucléaire (%),TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%),PROD,CONSO,Année,Mois,Année_mois
0,11,Île-de-France,2023-02-01,2,938289.0000,40625.0000,1082217.3010,4588.0000,1327.0000,576.0000,19056.0000,16071.0000,875139.0000,4.3733,19.1475,124.9227,89.4917,0.4935,37.6313,0.1309,7.5952,0.0630,30.0000,1.7359,52.3155,1145404.3010,957345.0000,2023,2,2023-02
1,11,Île-de-France,2023-02-02,3,930200.0000,47379.0000,1079019.0727,3857.0000,1032.0000,576.0000,71808.0000,16013.0000,861368.0000,5.1579,22.3314,130.6891,89.1478,0.4308,31.6353,0.1029,5.9071,0.0641,30.0000,1.7441,52.1269,1147876.0727,1002008.0000,2023,2,2023-02
2,11,Île-de-France,2023-02-03,4,894875.0000,47413.0000,1077740.8443,2611.0000,1134.0000,576.0000,41664.0000,16389.0000,826787.0000,5.3769,22.3479,140.3549,89.0479,0.3016,21.4158,0.1154,6.4908,0.0660,30.0000,1.8570,53.3510,1145863.8443,936539.0000,2023,2,2023-02
3,11,Île-de-France,2023-02-04,5,814953.0000,47455.0000,1072644.0415,1366.0000,1112.0000,576.0000,15072.0000,16596.0000,747869.0000,5.8986,22.3669,138.1038,88.6635,0.1710,11.2046,0.1253,6.3650,0.0717,30.0000,2.0628,54.0241,1139749.0415,830025.0000,2023,2,2023-02
4,11,Île-de-France,2023-02-05,6,806467.0000,47389.0000,1076077.9516,4219.0000,1756.0000,576.0000,3984.0000,16509.0000,736050.0000,5.9609,22.3358,134.8447,88.8743,0.5100,34.6045,0.2083,10.0507,0.0721,30.0000,2.0766,53.7411,1146526.9516,810451.0000,2023,2,2023-02


In [25]:
# pour enregistrer et télécharger le fichier en local, pour garder une trace de la progression.
df_clean.to_csv("eco2mix_clean_2.csv", index=False)

In [27]:
# FACULTATIF
from google.colab import files
files.download("eco2mix_clean_2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>